In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

# Общая информация о данных

In [ ]:
train_df = pd.read_json('../input/two-sigma-connect-rental-listing-inquiries/train.json.zip').reset_index(drop=True)
test_df = pd.read_json('../input/two-sigma-connect-rental-listing-inquiries/test.json.zip').reset_index(drop=True)

Посмотрим на тренировочный датасет.

In [ ]:
train_df.head()

In [ ]:
train_df.info()

# Предобработка данных

## Тренировочный датасет

Для того, чтобы информация из переменной features далее могла быть использована при построении модели, выполним преобразование:

In [ ]:
train_df['len_features'] = train_df['features'].apply(len)

Изменим формат переменной created, полученные значения будем использовать для создания трех столбцов: год, месяц и день.

In [ ]:
train_df['time'] = pd.to_datetime(
    train_df['created'], format='%Y-%m-%dT%H:%M:%S')
train_df['year'] = pd.DatetimeIndex(train_df['time']).year
train_df['month'] = pd.DatetimeIndex(train_df['time']).month
train_df['weekday'] = pd.DatetimeIndex(train_df['time']).weekday

In [ ]:
train_df.info()

Удалим столбцы в датасете, которые не будут участвовать при построении модели.

In [ ]:
train_df = train_df.drop(['building_id', 'description', 'display_address', 'features', 'listing_id', 'manager_id', 'photos', 'street_address', 'created', 'time'], axis = 1)

Посмотрим на распределение переменных в датасете. Построим гистограммы для переменных: bathrooms, bedrooms.

In [ ]:
plt.hist(x = train_df.bathrooms, bins = 6, alpha = 0.5, color = 'green')
plt.title('bathrooms')
plt.xlabel('количество ванных комнат')
plt.ylabel('количество наблюдений')
plt.grid(visible = True)

Для того, чтобы сделать прогноз целевой переменной, выполним ее категоризацию.

In [ ]:
train_df.interest_level.unique()

In [ ]:
train_df.interest_level.value_counts()

In [ ]:
def interest_level_category(row): 
    value = row['interest_level']   
    if value == 'low':
        return 0
    elif value == 'medium':
        return 1
    else:
        return 2

In [ ]:
train_df['interest_level'] = train_df.apply(interest_level_category, axis = 1)
train_df.interest_level.value_counts()

Изучим распределение целевой переменной.

In [ ]:
plt.hist(x = train_df.interest_level, bins = 3, alpha = 0.5, color = 'blue')
plt.title('interest_level')
plt.xlabel('interest_level')
plt.ylabel('количество наблюдений')
plt.grid(visible = True)

По полученному графику можно увидеть, что наиболее часто встречается первая категория  - low, наиболее редко - high.

## Тестовый датасет

Выполним аналогичные преобразования для тестового датасета: преобразование переменных features и time, удаление переменных.

In [ ]:
test_df['len_features'] = test_df['features'].apply(len)

In [ ]:
test_df['time'] = pd.to_datetime(
    test_df['created'], format='%Y-%m-%dT%H:%M:%S')
test_df['year'] = pd.DatetimeIndex(test_df['time']).year
test_df['month'] = pd.DatetimeIndex(test_df['time']).month
test_df['weekday'] = pd.DatetimeIndex(test_df['time']).weekday

In [ ]:
test_df = test_df.drop(['building_id', 'description', 'display_address', 'features', 
                          'manager_id', 'photos', 'street_address', 'created', 'time'], axis = 1)

# Построение модели

Разделим train_df на обучающую (60 %), валидационную (20 %) и тестовую выборки (20 %).

In [ ]:
train, validate, test = np.split(train_df, [int(.6*len(train_df)), int(.8*len(train_df))])

In [ ]:
train_target = train.interest_level
train_features = train.drop('interest_level', axis = 1)

validate_target = validate.interest_level
validate_features = validate.drop('interest_level', axis = 1)

test_target = test.interest_level
test_features = test.drop('interest_level', axis = 1)

In [ ]:
params = [ {'n_estimators' : list(range(10,51,10)), 'max_depth': list(range(1,13,1))}] 

In [ ]:
clf = GridSearchCV(RandomForestClassifier(), param_grid = params, cv = 5, verbose=True, n_jobs=-1)

In [ ]:
best_clf = clf.fit(train_features, train_target)

In [ ]:
result_validate= best_clf.score(validate_features, validate_target)
result_test = best_clf.score(test_features, test_target) 

print("Accuracy модели на валидационной выборке:", result_validate)
print("Accuracy модели на тестовой выборке:", result_test)

In [ ]:
features = test_df.drop(['listing_id'], axis = 1)

In [ ]:
features.head()

In [ ]:
sub_result = best_clf.predict_proba(features)

In [ ]:
target_num = {'low':0, 'medium':1, 'high':2}

In [ ]:
submission = pd.DataFrame()
submission["listing_id"] = test_df["listing_id"]
for label in ["high", "medium", "low"]:
    submission[label] = sub_result[:, target_num[label]]
submission.to_csv("submission.csv", index=False)